In [1]:
from bs4 import BeautifulSoup
headers = {"User-Agent": "Chrome/98.0.4758.102"}
#r = requests.get(url, headers=headers)
import pandas as pd
import requests
import ipywidgets as widgets
from ipywidgets import interact
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import lxml
from lxml import html



requests = requests.Session()
retry = Retry(connect=3, backoff_factor=0.6)
adapter = HTTPAdapter(max_retries=retry)
requests.mount('http://', adapter)
requests.mount('https://', adapter)



In [2]:
try:
    '''
    Requesting data from a given URL and saving it as text
    '''
    web_page = requests.get('https://finance.yahoo.com/quote/MMM/key-statistics',headers=headers).text
    soup = BeautifulSoup(web_page, "html.parser")
    print('Success')
except:
    print('Could not get URL')
    
   

Success


In [3]:
# Parse the page with LXML, so that we can start doing some XPATH queries
# to extract the data that we want
tree = html.fromstring(web_page)
company_name = tree.xpath("//h1/text()")
# Smoke test that we fetched the page by fetching and displaying the H1 element

company_name


['3M Company (MMM)']

In [4]:
# Scraping financial ststistics for the company
fin_stat = soup.find('td', {'class':'Fw(500) Ta(end) Pstart(10px) Miw(60px)'})
print(fin_stat)

<td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)">0.96</td>


In [5]:
fin_stat_data = fin_stat.text
print(fin_stat_data)

0.96


In [6]:
# Scraping the  financial statistics 
elements = []
stats = soup.find_all('td', {'class':'Fw(500) Ta(end) Pstart(10px) Miw(60px)'}) #('div', {'class':'Mb(10px) Pend(20px) smartphone_Pend(0px) '})
for stat in stats:
    elements.append(stat.text)
print(elements)

['0.96', '-14.68%', '12.37%', '208.95', '139.74', '168.75', '184.73', '3.07M', '4.85M', '571.1M', 'N/A', '570.53M', '0.12%', '68.48%', '9.86M', '3.25', '1.73%', '1.72%', '9.31M', '5.96', '3.96%', '5.92', '4.12%', '2.96', '58.50%', 'Mar 11, 2022', 'Feb 16, 2022', '2:1', 'Sep 29, 2003', 'Dec 30, 2021', 'Dec 30, 2021', '16.75%', '22.03%', '10.31%', '42.28%', '35.36B', '61.06', '0.30%', '16.58B', '9.7B', '5.92B', '10.12', '-4.70%', '4.77B', '8.34', '18.32B', '121.17', '1.71', '26.31', '7.45B', '5.04B']


##-Shares% Held by Insiders (Insider_Share_Perct) is the 12th element #pandas starts counting from 0, thus 12 instead of 13th element
##-Shares% Held by Institutions (Institutions_Share_Perct) is the 13th element
##Profit Margin (16.75%) is the 31st element
##Operating Margin (ttm) is the 32nd element
##Return on Assets (ttm) is the 33rd element
##Return on Equity (ttm) is the 34th element
##Revenue (ttm) is the 35th element
##Gross Profit (ttm) is the 38th element
##EBITDA is the 39th element


In [7]:
df = pd.DataFrame({'Insider_Share_Perct': elements[12], #pandas starts counting from 0, thus 12 instead of 13th element
                 'Institutions_Share_Perct': elements[13], 
                 'Profit Margin': elements[31], 
                 'Operating Margin': elements[32], 
                 'Return on Assets': elements[33],
                  'Return on Equity': elements[34],
                  'Revenue': elements[35],
                  'Gross Profit': elements[38],
                  'EBITDA': elements[39]}, 
                 index = ['MMM'])
df

,Insider_Share_Perct,Institutions_Share_Perct,Profit Margin,Operating Margin,Return on Assets,Return on Equity,Revenue,Gross Profit,EBITDA
MMM,0.12%,68.48%,16.75%,22.03%,10.31%,42.28%,35.36B,16.58B,9.7B


In [8]:
SPFirms_Ticker = pd.read_csv('SPFirms_Ticker.csv')
SPFirms_Ticker

,Ticker
0,MMM
1,AOS
2,ABT
3,ABBV
4,ABMD
...,...
499,YUM
500,ZBRA
501,ZBH
502,ZION


In [9]:
#get rest of S&P data
tickers = SPFirms_Ticker['Ticker']
type(tickers)


pandas.core.series.Series

In [10]:
tickers.head

<bound method NDFrame.head of 0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
499     YUM
500    ZBRA
501     ZBH
502    ZION
503     ZTS
Name: Ticker, Length: 504, dtype: object>

In [11]:
# Parse the page with LXML, so that we can start doing some XPATH queries
# to extract the data that we want
tree = html.fromstring(web_page)
company_name = tree.xpath("//h1/text()")
# Smoke test that we fetched the page by fetching and displaying the H1 element
#company_name = tree.xpath("//h1/text()")
company_name


['3M Company (MMM)']

In [12]:
def financial_statistics(ticker):
    '''
    Scraping for the ESG scores for a company and appending them to a dataframe.
    
    
    This function takes a ticker (firm's abbreviation) as input and then generate the Yahoo! Finace Sustainability URL. 
    If the URL is valid, BeautifulSoup will then parse the text and save the desired data points, 
    All data then will be saved in a dataframe.
    '''
    elements = []        
   
    web_page = requests.get('https://finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker, headers=headers).text
    
    soup = BeautifulSoup(web_page, "html.parser")
    
    stats = soup.find_all('td', {'class':'Fw(500) Ta(end) Pstart(10px) Miw(60px)'}) #('div', {'class':'Mb(10px) Pend(20px) smartphone_Pend(0px) '})
    
    # Parse the page with LXML, so that we can start doing some XPATH queries to extract company name data that we want
    tree = html.fromstring(web_page)
    # Smoke test that we fetched the page by fetching and displaying the H1 element
    company_name = tree.xpath("//h1/text()")
    for stat in stats:
        elements.append(stat.text)
    df = pd.DataFrame({'Insider_Share_Perct': elements[12], 
                 'Institutions_Share_Perct': elements[13], 
                 'Profit Margin': elements[31], 
                 'Operating Margin': elements[32], 
                 'Return on Assets': elements[33],
                  'Return on Equity': elements[34],
                  'Revenue': elements[35],
                  'Gross Profit': elements[38],
                  'EBITDA': elements[39]}, 
                 index = [ticker])
    for i in tickers:
        df['Company_name']= company_name
    
    return df

In [13]:


financial_data = pd.DataFrame({'Insider_Share_Perct':[],
                 'Institutions_Share_Perct':[], 
                 'Profit Margin': [], 
                 'Operating Margin': [], 
                 'Return on Assets': [],
                  'Return on Equity': [],
                  'Revenue': [],
                  'Gross Profit': [],
                  'EBITDA': [],
                  'Company_name': []})

'''
    Runs the yahoo_finance() function for each company in the tickers series and writes all the data in a consolidated dataframe.
    
    If the data is not retrieved successfully, print no data for + ticker.
'''
    #time.sleep(6)
for i in tickers:
    individual_record = financial_statistics(i)
    if individual_record is None:
        financial_statistics(i) + 1  # check Progress
    else: 
        print(i + " retrieved data")#  check Progress
        financial_data = financial_data.append(individual_record)
print(financial_data)


MMM retrieved data
AOS retrieved data
ABT retrieved data
ABBV retrieved data
ABMD retrieved data
ACN retrieved data
ATVI retrieved data
ADM retrieved data
ADBE retrieved data
ADP retrieved data
AAP retrieved data
AES retrieved data
AFL retrieved data
A retrieved data
AIG retrieved data
APD retrieved data
AKAM retrieved data
ALK retrieved data
ALB retrieved data
ARE retrieved data
ALGN retrieved data
ALLE retrieved data
LNT retrieved data
ALL retrieved data
GOOGL retrieved data
GOOG retrieved data
MO retrieved data
AMZN retrieved data
AMCR retrieved data
AMD retrieved data
AEE retrieved data
AAL retrieved data
AEP retrieved data
AXP retrieved data
AMT retrieved data
AWK retrieved data
AMP retrieved data
ABC retrieved data
AME retrieved data
AMGN retrieved data
APH retrieved data
ADI retrieved data
ANSS retrieved data
ANTM retrieved data
AON retrieved data
APA retrieved data
AAPL retrieved data
AMAT retrieved data
APTV retrieved data
ANET retrieved data
AIZ retrieved data
T retrieved dat

SBUX retrieved data
STT retrieved data
STE retrieved data
SYK retrieved data
SIVB retrieved data
SYF retrieved data
SNPS retrieved data
SYY retrieved data
TMUS retrieved data
TROW retrieved data
TTWO retrieved data
TPR retrieved data
TGT retrieved data
TEL retrieved data
TDY retrieved data
TFX retrieved data
TER retrieved data
TSLA retrieved data
TXN retrieved data
TXT retrieved data
TMO retrieved data
TJX retrieved data
TSCO retrieved data
TT retrieved data
TDG retrieved data
TRV retrieved data
TRMB retrieved data
TFC retrieved data
TWTR retrieved data
TYL retrieved data
TSN retrieved data
UDR retrieved data
ULTA retrieved data
USB retrieved data
UAA retrieved data
UA retrieved data
UNP retrieved data
UAL retrieved data
UNH retrieved data
UPS retrieved data
URI retrieved data
UHS retrieved data
VLO retrieved data
VTR retrieved data
VRSN retrieved data
VRSK retrieved data
VZ retrieved data
VRTX retrieved data
VFC retrieved data
VTRS retrieved data
V retrieved data
VNO retrieved data
VM

In [18]:
financial_data.to_csv('SPfinancial_datas.csv')